In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import csv
import copy
import sklearn.linear_model
import json
import re
from sklearn import preprocessing
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
COLOR_TREAT = "#2ecc71"
COLOR_NO_TREAT = "#e74c3c"
import warnings
warnings.filterwarnings('ignore')

In [2]:
pa = pd.read_excel('data/ucdp-peace-agreements.xls')

In order to prepare the table that will be used to display the peace agreements on our map, we clean the peace agreement dataset. First we chose to keep specific features in the table. 

**Peace agreement identification :**
- PAID : Peace Agreement ID
- pa_name : Peace Agreement name
- CID : Conflict ID
- Name : Conflict Name
- GWNO : Country code

**Peace agreement characteristics :**
- Inc : incompability (Territory, Government, Government/Territory)
- pa_date : PA signature date
- ended
- duration : date when PA is ended

**Comments on the peace agreement :**
- pa_comment : comments on the pa
- c_duration :  comment on agreement duration
- Link to fulltext agreement : pdf link

**Peace agreement type characteristics :**
- Mil_prov : Behavioral conduct of the warring parties
- Pol_prov : Regulation of governmental incompatibility
- Terr_prov : Regulation of territorial incompatibility
- Justice_prov : Justice issues




**Reference : **
Högbladh, Stina, 2011. ”Peace agreements 1975-2011 - Updating the UCDP Peace Agreement dataset”, in Pettersson Therése & Lotta Themnér (eds.), 2012, States in Armed Conflict 2011, Uppsala University: Department of Peace and Conflict Research Report 99
Harbom, Lotta, Stina Högbladh and Peter Wallensteen. 2006. “Armed Conflict and Peace Agreements.” Journal of Peace Research 43(5)

In [10]:
cols = ['PAID', 'GWNO', 'CID', 'Name', 'Inc', 'pa_name', 'pa_date ', 'pa_comment', 'ended', 'Duration', 'c_duration', 
        'Link to fulltext agreement', 'Mil_prov', 'Pol_prov', 'Terr_prov', 'Justice_prov']

In [11]:
pa_clean = pa[cols]
pa_clean.head(60)

,PAID,GWNO,CID,Name,Inc,pa_name,pa_date,pa_comment,ended,Duration,c_duration,Link to fulltext agreement,Mil_prov,Pol_prov,Terr_prov,Justice_prov
0,5521975120101,552,122,Zimbabwe (Rhodesia): government,2,Declaration of intent,1975-12-01,Type: peace process agreement\n \nDuring the l...,1,1976-03-19 00:00:00,NaN,http://ucdp.uu.se/downloads/fullpeace/Zim%2019...,0,0,0,0
1,5521979122101,552,122,Zimbabwe (Rhodesia): government,2,Lancaster House Agreement,1979-12-21,Type: full peace agreement\n \nThe Lancaster H...,0,NaN,NaN,http://ucdp.uu.se/downloads/fullpeace/Zim%2019...,1,1,1,1
2,3451991071201,345,189,Yugoslavia: Slovenia,1,Brioni Agreement,1991-07-12,"Partial peace agreement On 7 July, the parties...",0,NaN,"The peace agreement was not broken, yet, its i...",http://ucdp.uu.se/downloads/fullpeace/Yug%2019...,1,0,0,1
3,3451999060301,345,218,Yugoslavia: Kosovo,1,Kosovo peace agreement 1,1999-06-03,The accord was not a peace agreement in the st...,0,NaN,No unambiguous assessment of the success of th...,http://ucdp.uu.se/downloads/fullpeace/Yug%2019...,1,1,1,1
4,2001998041001,200,119,United Kingdom: Northern Ireland,1,The Good Friday Agreement,1998-04-10,Partial peace agreementAfter a long series of ...,0,NaN,The peace agreement cannot be considered as en...,http://ucdp.uu.se/downloads/fullpeace/UK%20199...,1,0,1,1
5,5001985121701,500,118,Uganda: government,2,Nairobi Peace Agreement,1985-12-17,Full agreement.\n \nThe Nairobi peace agreemen...,1,1986-01-13 00:00:00,NaN,http://ucdp.uu.se/downloads/fullpeace/Uga%2019...,1,1,0,1
6,1181988060301,500,118,Uganda: government,2,Gulu Peace Accord (Pece Peace Agreement),1988-06-03,"Full agreement\n \nThe agreement, which is som...",0,NaN,NaN,http://ucdp.uu.se/downloads/fullpeace/Uga%2019...,1,1,0,1
7,5002002122401,500,118,Uganda: government,2,Yumbe Peace Agreement,2002-12-24,Full peace agreement\nThe accord provides for ...,0,NaN,NaN,http://ucdp.uu.se/downloads/fullpeace/Uga%2020...,1,1,0,1
8,5002007050201,500,118,Uganda: government,2,Agreement on Comprehensive Solutions between t...,2007-05-02,Partial peace agreement\n \nThe Agreement on C...,1,2008-09-11,This agreement was concluded as part of the Ju...,http://ucdp.uu.se/downloads/fullpeace/Uga%2020...,1,0,1,1
9,5002007062901,500,118,Uganda: government,2,Agreement on Accountability and Reconciliation...,2007-06-29,Partial peace agreement\n \nThe Agreement on A...,1,2008-09-11,This agreement was concluded as part of the Ju...,http://ucdp.uu.se/downloads/fullpeace/Uga%2020...,0,0,0,1


In [12]:
reader = csv.reader(open('data/GWNO.csv', 'r'))
#Create a dictionnary that will have {New ID : Old ID} so we can make the link
d = {}
for row in reader:
    k, v = row
    d[k] = v

In [13]:
def clean2sides(sides):
    if ',' in sides:
        codec = ''
        for side in sides.split(', '):
            codec += d[side] + ','
        return codec[:-1]
    else:
        sides = re.sub(' ', '', sides)
        return d[sides]

In [16]:
#pa_clean['location'] = pa_clean.GWNO.astype(dtype=str).replace(to_replace=d )
pa_clean['location'] = pa_clean.GWNO.astype(dtype=str).apply(clean2sides)

In [17]:
pa_clean.drop(columns='GWNO', inplace=True)


,PAID,GWNO,CID,Name,Inc,pa_name,pa_date,pa_comment,ended,Duration,c_duration,Link to fulltext agreement,Mil_prov,Pol_prov,Terr_prov,Justice_prov,location
0,5521975120101,552,122,Zimbabwe (Rhodesia): government,2,Declaration of intent,1975-12-01,Type: peace process agreement\n \nDuring the l...,1,1976-03-19 00:00:00,NaN,http://ucdp.uu.se/downloads/fullpeace/Zim%2019...,0,0,0,0,Zimbabwe
1,5521979122101,552,122,Zimbabwe (Rhodesia): government,2,Lancaster House Agreement,1979-12-21,Type: full peace agreement\n \nThe Lancaster H...,0,NaN,NaN,http://ucdp.uu.se/downloads/fullpeace/Zim%2019...,1,1,1,1,Zimbabwe
2,3451991071201,345,189,Yugoslavia: Slovenia,1,Brioni Agreement,1991-07-12,"Partial peace agreement On 7 July, the parties...",0,NaN,"The peace agreement was not broken, yet, its i...",http://ucdp.uu.se/downloads/fullpeace/Yug%2019...,1,0,0,1,Yugoslavia
3,3451999060301,345,218,Yugoslavia: Kosovo,1,Kosovo peace agreement 1,1999-06-03,The accord was not a peace agreement in the st...,0,NaN,No unambiguous assessment of the success of th...,http://ucdp.uu.se/downloads/fullpeace/Yug%2019...,1,1,1,1,Yugoslavia
4,2001998041001,200,119,United Kingdom: Northern Ireland,1,The Good Friday Agreement,1998-04-10,Partial peace agreementAfter a long series of ...,0,NaN,The peace agreement cannot be considered as en...,http://ucdp.uu.se/downloads/fullpeace/UK%20199...,1,0,1,1,United Kingdom
5,5001985121701,500,118,Uganda: government,2,Nairobi Peace Agreement,1985-12-17,Full agreement.\n \nThe Nairobi peace agreemen...,1,1986-01-13 00:00:00,NaN,http://ucdp.uu.se/downloads/fullpeace/Uga%2019...,1,1,0,1,Uganda
6,1181988060301,500,118,Uganda: government,2,Gulu Peace Accord (Pece Peace Agreement),1988-06-03,"Full agreement\n \nThe agreement, which is som...",0,NaN,NaN,http://ucdp.uu.se/downloads/fullpeace/Uga%2019...,1,1,0,1,Uganda
7,5002002122401,500,118,Uganda: government,2,Yumbe Peace Agreement,2002-12-24,Full peace agreement\nThe accord provides for ...,0,NaN,NaN,http://ucdp.uu.se/downloads/fullpeace/Uga%2020...,1,1,0,1,Uganda
8,5002007050201,500,118,Uganda: government,2,Agreement on Comprehensive Solutions between t...,2007-05-02,Partial peace agreement\n \nThe Agreement on C...,1,2008-09-11,This agreement was concluded as part of the Ju...,http://ucdp.uu.se/downloads/fullpeace/Uga%2020...,1,0,1,1,Uganda
9,5002007062901,500,118,Uganda: government,2,Agreement on Accountability and Reconciliation...,2007-06-29,Partial peace agreement\n \nThe Agreement on A...,1,2008-09-11,This agreement was concluded as part of the Ju...,http://ucdp.uu.se/downloads/fullpeace/Uga%2020...,0,0,0,1,Uganda
